In [ ]:
import pandas as pd
import openai
import requests

#######################################
# Set your API key and data path
PATH = "./DATA/"

## OpenAI API Key
# valid for this education session only
api_key = "[API_KEY]"

# Data download from github and save to PATH
!wget -P $PATH https://raw.githubusercontent.com/leelabsg/KOGO_OpenAI_API_Tutorial_2025/main/Session1/note1_ShorthandStyle.txt
!wget -P $PATH https://raw.githubusercontent.com/leelabsg/KOGO_OpenAI_API_Tutorial_2025/main/Session1/note2_MIMICstyle.txt


## Check Clinical Note Examples

**1) Clinical Note 불러 오는 방법 안내**

파일에서 Clinical Note 불러오기

- 텍스트(.txt) 파일에 작성된 노트를 불러오는 방식입니다.
- 불러올 파일 경로를 정확히 지정해야 합니다.


**2) 예시로 사용되는 note1과 note2는 서로 다른 내용입니다:**
- note1: shorthand 스타일의 간단한 요약형 노트
- note2: narrative 스타일의 서술형 노트로, 보다 상세한 문맥과 진단 경과가 포함되어 있습니다

이 두 노트를 각각 사용하여 prompt 차이, 언어 차이, 노트 형식 차이에 따른 분석 결과 차이를 실험해볼 수 있습니다.

### 1) Shorthand version

In [2]:
## (1) Load note from .txt file
with open(f"{PATH}note1_ShorthandStyle.txt", "r", encoding="utf-8") as f: 
    note1 = f.read()

In [3]:
## check loaded note
print(note1)

#. T2DM  
   Dx since 2010, recent HbA1c 9.1  
   MTF → 설사, 복부팽만, 식욕저하, 무기력감  
   Lactic acidosis (Lac 3.9) 의심 → 약 중단, 수액 치료  
   MTF-induced lactic acidosis 의심 → 약 중단 후 회복, basal insulin 예정

   Levemir 22 --- 10 units, 저녁 인슐린 잘 안 맞음


#. HTN  
   BP on admission 92/58
   Losartan 유지 

#. CKD stage 3  
   Baseline Cr 1.6 → 입원 시 Cr 2.1  
   수액 이후 신기능 호전, 투석 필요 없음  

#. Dyslipidemia  
   Atorvastatin  



### 2) Narrative version (MIMIC-style) 

In [4]:
## (1) Load note from .txt file
with open(f"{PATH}note2_MIMICstyle.txt", "r", encoding="utf-8") as f: 
    note2 = f.read()

In [5]:
## check loaded note
print(note2[:500])  # 앞 500자만 출력

History of Present Illness:  
78-year-old female with a history of type 2 diabetes, hypertension, and CKD stage 3 who presented with generalized weakness, poor oral intake, and nausea.

Two weeks prior to admission, she was started on metformin after a primary care visit revealed HbA1c of 9.1. Since initiation, the patient has developed persistent loose stools and intermittent abdominal cramping. No reported fever or vomiting. Her family noted a gradual decline in appetite and energy over the pa


### 본 실습에서는 note1 (shorthand version)을 사용합니다. 만약 narrative version을 테스트하고 싶으면 아래의 코드를 수정하면 됩니다.

In [6]:
note = note1

## 1. Setting OpenAI API

In [7]:
client = openai.OpenAI(api_key=api_key) 

## [Task 1] Drug Name Extraction and drug code identification

이 Task에서는 임상노트로부터 언급된 약물명을 추출하는 작업을 수행합니다.

- GPT에게 임상노트를 입력하고, 텍스트에서 **약물명** 추출 및 약물에 상응하는 **ATC CODE**를 추출하게 합니다.
- 약물은 **일반명** 또는 **약어**(예: MFM = metformin)로 표현될 수 있습니다.



### (1) Drug name only 

In [8]:
prompt1_eng = f"""

Task: Extract all drug names mentioned in the following clinical note.
- Drug names may be written as abbreviations.
- Return the list of drug names only, without any additional explanation.

Clinical Note:
{note}
"""

In [9]:
## English Prompt Version
model="gpt-5"

response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "You are a clinical text extraction assistant."},
        {"role": "user", "content": prompt1_eng}
    ]
)

print(response.choices[0].message.content)

MTF
Levemir
insulin
Losartan
Atorvastatin


### (2) Drug name+description+code

In [10]:
prompt2_eng = f"""

Task: Extract all drug names mentioned in the following clinical note.
- Drug names may be written as abbreviations.
- Return the list of drug names, simple descriptions, and corresponding ATC codes in a table format (CSV).

Clinical Note:
{note}
"""

model="gpt-5"

response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "You are a clinical text extraction assistant."},
        {"role": "user", "content": prompt2_eng}
    ]
)


print(response.choices[0].message.content)

Drug Name,Description,ATC Code
Metformin (MTF),Biguanide antihyperglycemic for type 2 diabetes,A10BA02
Levemir (insulin detemir),Long-acting basal insulin,A10AE05
Losartan,Angiotensin II receptor blocker for hypertension and kidney protection,C09CA01
Atorvastatin,HMG-CoA reductase inhibitor (statin) for dyslipidemia,C10AA05


### Usage check 
사용한 토큰을 아래의 코드를 이용해서 체크할 수 있습니다.

In [11]:
print(response.usage)

CompletionUsage(completion_tokens=1448, prompt_tokens=251, total_tokens=1699, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=1344, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))


## [Task 2] Adverse Drug Reaction Detection

이 Task에서는 임상노트에 언급된 **약물 부작용(Adverse Drug Reactions, ADR)** 을 탐지하는 작업을 수행합니다.

- GPT에게 임상노트를 입력하고, 특정 약물 투여 이후 발생한 부작용을 **약물-증상 쌍(drug → symptom)** 형태로 추출하게 합니다.
- 약물명은 약어로 표현될 수 있으며, 부작용은 명시적 또는 암시적으로 표현되어 있을 수 있습니다.

**비교 실험:**
**Minimal Prompt** vs **Detailed Prompt** Prompt 구조에 따른 차이 비교
 

### Minimal Prompt vs Detailed Prompt

In [12]:
## Simple Prompt Ver.
simple_prompt2 = f"""

Task: Extract all possible adverse drug reactions (ADRs) mentioned in the clinical note.
- Return results in JSON format, with each item containing:
  - "drug": drug name
  - "symptom": symptom or adverse effect
  - "evidence": supporting sentence or phrase from the note
- No extra explanation.

Clinical Note:
{note}
"""

In [13]:
## Detailed Prompt Ver. (More Specific Criteria with Clinical Reasoning)
detailed_prompt2 = f"""

Your task is to detect and extract confirmed adverse drug reactions (ADRs) from the following clinical note.

ONLY extract ADRs if all of the following conditions are met:
1. A drug is clearly administered or mentioned.
2. A symptom appears *after* or **in relation to** that drug.
3. The symptom is more likely due to the drug than the patient's underlying conditions (e.g., CKD, diabetes, dehydration, infection).
4. The note provides sufficient context to support a reasonable causal relationship.

For each confirmed ADR, return a structured JSON object with the following fields:
- "drug": the name of the drug (e.g., Metformin)
- "symptom": the adverse effect or symptom (e.g., Lactic acidosis)
- "evidence": a direct quote from the note that supports the ADR (one sentence or phrase)
- "reason": a brief explanation of why this is likely an ADR and not due to other causes

If no ADR meets the above criteria, return an empty list.

Format your output as a JSON list.

Clinical Note:
{note}
"""

In [14]:
## English Prompt Version
response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "You are a clinical text extraction assistant."},
        {"role": "user", "content": simple_prompt2}
    ]
)

print(response.choices[0].message.content)

[
  {
    "drug": "MTF",
    "symptom": "diarrhea",
    "evidence": "MTF → 설사"
  },
  {
    "drug": "MTF",
    "symptom": "abdominal bloating/distension",
    "evidence": "MTF → 복부팽만"
  },
  {
    "drug": "MTF",
    "symptom": "decreased appetite",
    "evidence": "MTF → 식욕저하"
  },
  {
    "drug": "MTF",
    "symptom": "lethargy/fatigue",
    "evidence": "MTF → 무기력감"
  },
  {
    "drug": "MTF",
    "symptom": "lactic acidosis",
    "evidence": "MTF-induced lactic acidosis 의심 → 약 중단 후 회복"
  }
]


In [15]:
## English Prompt Version
response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "You are a clinical text extraction assistant."},
        {"role": "user", "content": detailed_prompt2}
    ]
)

print(response.choices[0].message.content)

[
  {
    "drug": "Metformin",
    "symptom": "Diarrhea",
    "evidence": "MTF → 설사, 복부팽만, 식욕저하, 무기력감",
    "reason": "The note explicitly links diarrhea to metformin; GI upset is a known metformin adverse effect and is more likely than being caused by T2DM itself."
  },
  {
    "drug": "Metformin",
    "symptom": "Abdominal bloating",
    "evidence": "MTF → 설사, 복부팽만, 식욕저하, 무기력감",
    "reason": "The clinician attributes abdominal bloating to metformin; this is a typical metformin GI side effect rather than due to diabetes."
  },
  {
    "drug": "Metformin",
    "symptom": "Decreased appetite",
    "evidence": "MTF → 설사, 복부팽만, 식욕저하, 무기력감",
    "reason": "Decreased appetite is listed as occurring with metformin use, supporting a drug-related effect over baseline disease."
  },
  {
    "drug": "Metformin",
    "symptom": "Fatigue",
    "evidence": "MTF → 설사, 복부팽만, 식욕저하, 무기력감",
    "reason": "Fatigue is grouped by the clinician as occurring with metformin, indicating a likely adverse effec

## [Task 3] Clinical Note Summarization

이 Task에서는 임상노트에 포함된 핵심 정보를 GPT를 통해 한글로 **요약(summarization)** 하는 작업을 수행합니다.
- 임상노트에는 환자의 병력, 진단, 투약, 검사 결과, 치료 계획 등의 정보가 포함되어 있으며, 종종 매우 길고 복잡합니다.
- 이 Task는 환자에 대한 중요한 정보를 요약하여, 환자/보호자 설명에 활용될 수 있는 간결한 요약문을 생성하는 것이 목적입니다.
- 출력은 한국어 요약문으로 생성됩니다.



In [16]:
prompt3_family = f"""


Task: Summarize the following clinical note in simple language for the patient’s family.
- Use non-technical, plain Korean language.
- The summary should explain what happened, what was diagnosed, how it was treated, and what to expect.
- Avoid complex terminology and abbreviations.

Output language: Korean

Clinical Note:
{note}
"""

In [ ]:
## Family-Friendly Summary
response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "You are a clinical text summarization assistant."},
        {"role": "user", "content": prompt3_family}
    ]
)

print(response.choices[0].message.content)

다음은 가족분들이 이해하시기 쉽게 정리한 내용입니다.

무슨 일이 있었나요?
- 오랫동안 당뇨가 있었고, 최근에 혈당이 잘 조절되지 않았습니다.
- 드시던 당뇨약을 복용한 뒤 설사, 배가 더부룩함, 식욕 저하, 무기력감이 생겼습니다.
- 병원에 오셨을 때 혈압이 일시적으로 낮았고, 콩팥 기능 수치도 평소보다 나빠져 있었습니다.

무엇으로 판단했나요?
- 복용하던 당뇨약의 부작용으로, 몸에 산이 쌓이는 위험한 상태가 된 것으로 의심했습니다.
- 콩팥 기능은 원래 조금 약한 편인데, 탈수 등으로 더 나빠진 상태가 겹친 것으로 보였습니다.
- 콜레스테롤과 혈압 문제는 기존대로 관리 중입니다.

어떻게 치료했나요?
- 문제가 된 당뇨약은 바로 중단했습니다.
- 수액(정맥 주사로 물과 전해질 보충)을 하여 탈수와 몸 상태를 빠르게 교정했고, 증상이 호전되었습니다.
- 콩팥 기능도 수액 치료 후 좋아졌고, 투석은 필요하지 않았습니다.
- 앞으로는 알약 대신 하루에 한 번 맞는 인슐린 주사로 혈당을 조절하기로 했습니다.
- 혈압약과 콜레스테롤약은 계속 복용합니다.

앞으로 무엇을 기대하고, 무엇을 주의해야 하나요?
- 당뇨약(문제 되었던 약)은 다시 드시지 않습니다. 반드시 의료진과 상의 후에만 약을 변경하세요.
- 인슐린 주사를 매일 같은 시간에 빠뜨리지 않고 맞는 것이 중요합니다. 교육해 드린 방법대로 혈당을 자주 확인해 주세요.
- 평소보다 심한 설사, 구토, 심한 피로감, 숨이 가쁨, 어지러움, 소변이 아주 줄어듦 등이 있으면 바로 병원에 연락하세요.
- 물을 충분히 드시고, 규칙적인 식사와 가벼운 운동으로 혈당을 함께 관리하면 좋습니다.
- 외래에서 혈당 조절 상태와 콩팥 기능을 정기적으로 확인할 예정입니다. 예약 일정을 지켜 주세요.

요약
- 당뇨약 부작용으로 몸 상태가 나빠졌으나, 약을 중단하고 수액 치료 후 호전됨.
- 앞으로는 하루 한 번 인슐린 주사로 혈당을 관리.
- 콩팥 기능은 좋아졌고, 투석은 필요 없음.
- 혈압약과 콜레스테롤약은 계속 복용하며,